In [1]:
from tkinter import *
from datetime import datetime
from tkinter.ttk import Combobox
from tkinter import messagebox
import pandas as pd
from sklearn.neighbors import NearestNeighbors

df=pd.read_csv("c:/users/raju/downloads/movies_collaborative.csv")
user_movie_matrix=df.pivot_table(index='userId',columns='movieId',values='rating')
user_movie_matrix.fillna(0,inplace=True)
model=NearestNeighbors(metric='cosine')
model.fit(user_movie_matrix)
movieid=user_movie_matrix.columns 

win=Tk()
win.state('zoomed')
win.configure(bg='powder blue')

lbl=Label(win,text='Movie Recommendation',font=('arial',25,'bold'),bg='powder blue')
lbl.pack()

date_time=Label(win,text=f"{datetime.now().date()}",font=('arial',15,'bold'),bg='powder blue',fg='black')
date_time.place(relx=.9,rely=.1)

def home_screen():
    frm=Frame(win)
    frm.place(relx=0,rely=.15,relwidth=1,relheight=.85)
    frm.configure(bg="orange") 

    def login(e_user=None,e_pass=None):
        if(e_user!=None and e_pass!=None):
            user=e_user.get()
            pwd=e_pass.get()
        else:
            user="admin"
            pwd="admin"
        if(len(user)==0 or len(pwd)==0):
            messagebox.showwarning("validation","Please fill both fields")
            return
        else:
            if(user=="admin" or pwd=="admin"):
                frm.destroy()
                mainscreen()  

    lab_user=Label(frm,text="UserId",font=("arial",20,'bold'),bg="orange",fg="white")
    lab_user.place(relx=.3,rely=.2) 

    e_user=Entry(frm,font=("arial",20,'bold'),bd=2.5)
    e_user.place(relx=.4,rely=.2) 
    e_user.focus() 

    lab_pass=Label(frm,text="Password",font=("arial",20,'bold'),bg="orange",fg="white")
    lab_pass.place(relx=.3,rely=.3) 

    e_pass=Entry(frm,font=("arial",20,'bold'),bd=2.5)
    e_pass.place(relx=.4,rely=.3) 

    button=Button(frm,command=lambda:login(e_user,e_pass),text='login',font=("arial",20,'bold'),bd=2.5)
    button.place(relx=.54,rely=.4) 

def mainscreen():
    ifrm=Frame(win) 
    ifrm.place(relx=0,rely=.15,relwidth=1.2,relheight=.85)
    ifrm.configure(bg='orange') 

    def new1():
        ifrm.destroy()
        home_screen()

    def recommendation():
        dis,idx=model.kneighbors([user_movie_matrix.iloc[0]],n_neighbors=5)
        si=idx[0][1]
        movie_by_similar_user=user_movie_matrix.iloc[si]
        ti=int(cb_type.get())
        movie_by_target_user=user_movie_matrix.iloc[ti]

        frm=Frame(ifrm)
        frm.place(relx=.2,rely=.4,relwidth=.26,relheight=.35) 
        frm.configure(bg='white')

        lbl=Label(frm,text="Similar movie recommended for you",font=('arial',15,'bold'),bg='white',fg="green") 
        lbl.pack() 

        recommended_movies={}
        for m,t,s in zip(movieid,movie_by_target_user,movie_by_similar_user):
            if t==0 and s!=0:
                recommended_movies[m]=s 
        final_movies_tob_recom=[] 
        for k,v in recommended_movies.items():
            if k>=5:
                final_movies_tob_recom.append(k)

        res=df[df.movieId.isin(final_movies_tob_recom)].title.value_counts().head() 
        mv=""
        for m in res.index:
            mv=mv+"\n"+m
        lbl.configure(text="Similar Movies\n"+mv)

    cb_type=Combobox(win,values=movieid.tolist(),font=("arial",30,'bold')) 
    cb_type.current(0) 
    cb_type.place(relx=.26,rely=.3) 

    back=Button(ifrm,command=new1,text='Back',font=("arial",20,'bold'),bd=2.5) 
    back.place(relx=0,rely=0) 

    back=Button(ifrm,command=recommendation,text='Recommendation',font=("arial",15,'bold'),bd=2.5) 
    back.place(relx=.55,rely=.4) 
     
home_screen()
win.mainloop()